# LSTM

- ## Preliminaries

- ### Imports

In [1]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout,TimeDistributed
from keras.layers import LSTM,SimpleRNN
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import os, os.path
from os import listdir
from os.path import isfile, join
from unicodedata import normalize
import re

Using TensorFlow backend.
/home/duc-vinh/anaconda2/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


- ### Check GPU usage

In [2]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

In [3]:
get_available_gpus()

[u'/gpu:0']

----------

- ### Check and set Twitter's API

In [4]:
import twitter
api = twitter.Api(consumer_key='LyNVanTEQEOEGKfXAMeLv6AKG',
                    consumer_secret='0lJvhaaOP5cRZWm6rxwyBIAypd1P7eiDx9f74KBDlLrSldNuBQ',
                    access_token_key='855852332034265088-geTEVmA7xIsOD3WCZyfBNnqjRdS1MhW',
                    access_token_secret='kJMwMl67e3nYrqaGWzIizxzQpRZhtBfOnwPflO1fk3cOt')

/home/duc-vinh/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-04-27 20:45:00.084187. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [5]:
print(api.VerifyCredentials())

{"created_at": "Sat Apr 22 18:34:31 +0000 2017", "default_profile": true, "description": "Learning how to be creative", "followers_count": 2, "friends_count": 1, "id": 855852332034265088, "lang": "en", "location": "Somewhere in the cloud", "name": "ArtistBot", "profile_background_color": "F5F8FA", "profile_banner_url": "https://pbs.twimg.com/profile_banners/855852332034265088/1492892354", "profile_image_url": "http://pbs.twimg.com/profile_images/855878764143804417/r55Z2Js5_normal.jpg", "profile_link_color": "1DA1F2", "profile_sidebar_fill_color": "DDEEF6", "profile_text_color": "333333", "screen_name": "TheTalkativeBot", "status": {"created_at": "Tue Apr 25 22:31:31 +0000 2017", "id": 856999137753145349, "id_str": "856999137753145349", "in_reply_to_screen_name": "TheTalkativeBot", "in_reply_to_status_id": 856894659007844352, "in_reply_to_user_id": 855852332034265088, "lang": "en", "media": [{"display_url": "pic.twitter.com/lMV1EeegY0", "expanded_url": "https://twitter.com/TheTalkativeB

/home/duc-vinh/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-04-27 20:45:00.534350. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


-------------------

# I. Learning from corpus

** 1. Load and convert data**

'''Example script to generate text from Nietzsche's writings.
At least 20 epochs are required before the generated text
starts sounding coherent.
It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.
If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

In [6]:
#Load and concatenate files:

DIR="../../LSTM/data/Gutenberg/ebooks-unzipped/German/"
all_files = [f for f in listdir(DIR) if isfile(join(DIR, f))]

#choose how many files to concatenate:
nb_files=7
if nb_files>len(all_files):
    nb_files=len(all_files)
    
    
out_path="german/data/"
if not os.path.exists(out_path):
    os.makedirs(out_path)
if not os.path.exists(out_path+"input/"):
    os.makedirs(out_path+"input/")
    
with open(out_path+'input/german.txt', 'w') as outfile:
    for fname in all_files[0:nb_files]:
        with open(DIR+fname) as infile:
            i=0
            for line in infile:
                if i>=50:
                    outfile.write(line)
                i=i+1
        print ("Done concatenating file : %s" %fname)

Done concatenating file : 2146-8.txt
Done concatenating file : 2779-8.txt
Done concatenating file : 2409-8.txt
Done concatenating file : 4013-8.txt
Done concatenating file : 2947-8.txt
Done concatenating file : 3498-8.txt
Done concatenating file : 2314-8.txt


/home/duc-vinh/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-04-27 20:45:00.978848. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [7]:
#load file
file_name=out_path+'input/german.txt'
text = open(file_name).read()
text=normalize('NFKD',text.decode('latin1')).encode('ASCII', 'ignore')

/home/duc-vinh/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-04-27 20:45:01.241985. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [8]:
#text = text.replace(to_delete,"").replace('Digitized by',"").replace('Google',"") 
text = re.sub("\n\n+" , "\n", text)

/home/duc-vinh/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-04-27 20:45:01.365770. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [9]:
print('corpus length:', len(text))

chars = sorted(list(set(text)))
VOCAB_SIZE = len(chars)
print('total chars:',VOCAB_SIZE)

corpus length: 2215861
total chars: 94


/home/duc-vinh/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-04-27 20:45:01.386392. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


**Warning:** The RNN takes in input numerical data hence the necessity to convert strings into numerical values.

In [10]:
#creating mapping between indexes and characters
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

/home/duc-vinh/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-04-27 20:45:01.509417. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


We’re gonna use Keras to create and train our Network, so we must convert the data into this form: (number_of_sequences, length_of_sequence, number_of_features).
- nb of features = length of the char array
- length of sequence = batch size
- nb of sequence = len(data) divided by batch size.

**Warning : ** target sequence is setted by shifting the source/input sequence by one character with both having the same length.

In [11]:
%%time

SEQ_LENGTH=100
#Build three dimensional arrays
X = np.zeros((len(text)/SEQ_LENGTH, SEQ_LENGTH, VOCAB_SIZE)) #input
y = np.zeros((len(text)/SEQ_LENGTH, SEQ_LENGTH, VOCAB_SIZE)) #target

#Build sequences
for i in range(0, len(text)/SEQ_LENGTH):
    X_sequence = text[i*SEQ_LENGTH:(i+1)*SEQ_LENGTH]
    X_sequence_ix = [char_indices[value] for value in X_sequence]
    input_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))
    for j in range(SEQ_LENGTH):
        input_sequence[j][X_sequence_ix[j]] = 1.
    X[i] = input_sequence

    y_sequence = text[i*SEQ_LENGTH+1:(i+1)*SEQ_LENGTH+1]
    y_sequence_ix = [char_indices[value] for value in y_sequence]
    target_sequence = np.zeros((SEQ_LENGTH, VOCAB_SIZE))
    for j in range(SEQ_LENGTH):
        target_sequence[j][y_sequence_ix[j]] = 1.
    y[i] = target_sequence

CPU times: user 2.06 s, sys: 204 ms, total: 2.26 s
Wall time: 2.27 s


/home/duc-vinh/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-04-27 20:45:01.576155. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


** 2. Build the network**

In [12]:
HIDDEN_DIM= 500 #500
LAYER_NUM = 2


model = Sequential()
model.add(LSTM(HIDDEN_DIM, input_shape=(None, VOCAB_SIZE), return_sequences=True))
for i in range(LAYER_NUM - 1):
    model.add(LSTM(HIDDEN_DIM, return_sequences=True))
model.add(TimeDistributed(Dense(VOCAB_SIZE)))
model.add(Activation('softmax'))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

/home/duc-vinh/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-04-27 20:45:04.503872. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [13]:
def generate_text(model, length, vocab_size, ix_to_char):
    # starting with random character
    ix = [np.random.randint(vocab_size)]
    y_char = [ix_to_char[ix[-1]]]
    X = np.zeros((1, length, vocab_size))
    for i in range(length):
        # appending the last predicted character to sequence
        X[0, i, :][ix[-1]] = 1
        print(ix_to_char[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(ix_to_char[ix[-1]])
    return ('').join(y_char)

/home/duc-vinh/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-04-27 20:45:09.047030. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [14]:
# Generate some sample before training to know how bad it is!
bla = generate_text(model, 100, VOCAB_SIZE, indices_char)
#api.PostUpdate(status=bla[0:123])

1Af}}k!<<<jjj---XSSS,d<<<jjj---Bdddddppp######IPZ     """1111b[[[lllxbb0nnn[[kOOOO####}}     """X7XN

/home/duc-vinh/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-04-27 20:45:09.055077. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


**3. Train network**

In [15]:
def get_iternb(string):
    return re.findall(r'checkpoint_500_epoch_(.*).hdf5', string)[0]

/home/duc-vinh/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-04-27 20:45:16.482911. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [16]:
#batch size equals to seq length here
BATCH_SIZE=300
#len of desired output
GENERATE_LENGTH=140
DIR=out_path+"weights/weight_attempt_s02/"
flag=True

try:
    onlyfiles = [f for f in listdir(DIR) if isfile(join(DIR, f))]
    iteration=[]
    for files in onlyfiles:
        iteration.append(int(get_iternb(files)))
    iteration=max(iteration)

    last_checkpoint=DIR+onlyfiles[0][0:21]+str(iteration)+'.hdf5'
except Exception as e:
    print(e)
    onlyfiles=[]
    if not os.path.exists(DIR):
        os.makedirs(DIR)
    nb_files=0

if nb_files>0:
    model.load_weights(last_checkpoint)
else:
    iteration=0
    
print("Starting at iteration : %s" %iteration)
while flag==True:
    print('\n')
    print('-'*20)
    model.fit(X, y, batch_size=BATCH_SIZE, verbose=2, nb_epoch=1)
    iteration += 1
    bla=generate_text(model, GENERATE_LENGTH,VOCAB_SIZE, indices_char)
    if iteration % 10 == 0:
        print("\n\nIteration nb : %s" %iteration)
        #api.PostUpdate(status=bla[0:123])
        model.save_weights(DIR+'checkpoint_{}_epoch_{}.hdf5'.format(HIDDEN_DIM, iteration))
        #remove unecessary files:
        for files in onlyfiles:
            try:
                if files:
                    os.remove(DIR+files)
            except:
                pass
        onlyfiles = [f for f in listdir(DIR) if isfile(join(DIR, f))]

    if iteration>=500:
        print("Stopping...")
        flag=False

Starting at iteration : 450


--------------------


/home/duc-vinh/anaconda2/lib/python2.7/site-packages/keras/models.py:834: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/1
258s - loss: 0.1558
$K. Welche ohne Schaffe sei nicht nach;
Sie sind die Lindenbuch, die zwei verbrennen!
Doch im nachtlichen Gehnal hat die krufte Haus,
Dru

--------------------
Epoch 1/1
256s - loss: 0.1242
) da ein Anderer, der sich die
Fehler dieser Vorganger zu Nutzen zu machen wite, wenig Muhe haben wurde, wenn er sich
befordern, nicht als

--------------------
Epoch 1/1
256s - loss: 0.1194
You provide, or agree to also provide on request at
          no additional cost, fee or expense, a copy of the
          etext in its ori

--------------------
Epoch 1/1
256s - loss: 0.1178
(welches auf eine oder die
andere Art doch allemal der stillschweigende Vorsatz einer jeden ist, die Krankunftigkeit
des Prinzen hatte ich

--------------------
Epoch 1/1
256s - loss: 0.1161





Aus der Harzreise
1824


Prolog

Schwarze Rocke, seidne Strumpfe,
Weie, hofliche Manspier die Zange machten
Mir fur den Stu

--------------------
Epoch 1/1
256s - loss: 0.1177
6. Der Lauf des Mondpfades

des weisen Spiel der Net

--------------------
Epoch 1/1
256s - loss: 0.1097
8. Und nun ist kurz den Herrn der Herrlichkeit, der in unserer Seele  sagte: "Und wenn ist es einst zu vermeiden,
welches ich verwundern, d

--------------------
Epoch 1/1
256s - loss: 0.1084
Geschaftspolitik, deren Wirkungen ich
erst spater und obfahrener mir zu lesen anfing, nablechte er den erstaunten, der in hohemaligen
Tage

--------------------
Epoch 1/1
256s - loss: 0.1075
Kapitel: Veranderung der Szene. Charakter der Syracusaner,
                    des Dionysius und seine Garbe,
wo Pilhaso naturhierte ihm m

--------------------
Epoch 1/1
255s - loss: 0.1083

Waagschale einer kalten unparteiischen uberlegung gemacht, und vermutlich
den entgegenstehenden Grundsatzen das Buch wir, so hatte er
wo

--------------------
Epoch 1/1
256s - loss: 0.1076
Ihr Mann
war stolz darauf.  Noch stolzer fast war ihre Mutter,
eine einfach gewohnte, sehr arbeitsam, ihne mir
menschlich man schon erzub

--------------------

/home/duc-vinh/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-04-27 20:45:16.487355. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


**4. Generate text**

In [17]:
def save_text(model, length, vocab_size, ix_to_char):
    # starting with random character
    ix = [np.random.randint(vocab_size)]
    y_char = [ix_to_char[ix[-1]]]
    X = np.zeros((1, length, vocab_size))
    for i in range(length):
        # appending the last predicted character to sequence
        X[0, i, :][ix[-1]] = 1
        print(ix_to_char[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(ix_to_char[ix[-1]])
    if not os.path.exists(out_path+"generate/"):
        os.makedirs(out_path+"generate/")
    with open(out_path+"generate/output.txt","w") as f:
        f.write(('').join(y_char))
    return ('').join(y_char)


/home/duc-vinh/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-04-28 00:26:37.364112. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [18]:
#seed with particular text:
def generate_text_seeded(model,seed,length, vocab_size, ix_to_char):
    # starting with random character
    # char_indices
    ix = [char_indices[x] for x in seed]
    y_char = [x for x in seed]
    X = np.zeros((1, length, vocab_size))
    for i in range(len(ix)) :
        X[0, i, :][ix[i]] = 1
        print(ix_to_char[ix[i]], end="")
    to_substract = len(ix)
    for i in range(length-to_substract):
        # appending the last predicted character to sequence
        X[0, i, :][ix[-1]] = 1
        print(ix_to_char[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(ix_to_char[ix[-1]])
    return ('').join(y_char)

/home/duc-vinh/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-04-28 00:26:37.375971. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [19]:
generate_text_seeded(model,normalize('NFKD',"ich bin ".decode('latin1')), 100, VOCAB_SIZE, indices_char)

ich bin  kh Alssirr nicht erwahnen.

Es ist ein schones Wort, das verrater schlafen,
Und was denn

u'ich bin kh Alssirr nicht erwahnen.\r\n\r\nEs ist ein schones Wort, das verrater schlafen,\r\nUnd was denn '

/home/duc-vinh/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-04-28 00:26:37.519624. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [20]:
%%time
out = save_text(model, 1500, VOCAB_SIZE, indices_char)

{
Freund by th' err of our etext refund of ther cannon by the Project Gutenberg legal advisor**

(Three Pages)


***START**THE SMALL PRINT!**FOR PUBLIC DOMAIN ETEXTS**START***
Why is this "Small Print!" statement here?  You know: lawyers.
They tell us you might sue us if there is something wrong with
your copy of this etext, even if you got it for free from
someone other than us, and even if what's wrong is not our
fault.  So, among other things, this "Small Print!" statement
disclaims most of our liability to you.  It also tells you how
you can distribute copies of this etext if you want to.

*BEFORE!* YOU USE OR READ THIS ETEXT
By using or reading any part of this PROJECT GUTENBERG-tm
etext, you indicate that you understand, agree to and accept
this "Small Print!" statement.  If you do not, you can receive
a refund of the money (if any) you paid for this etext by
sending a request within 30 days of receiving it to the person
you got it from.  If you received this etext on a physical


/home/duc-vinh/anaconda2/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-04-28 00:26:41.675267. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)
